# Installing the opendatasets api as a call to the Kaggle

In [1]:
!pip install opendatasets

import opendatasets as ods
ods.download("https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks")

Skipping, found downloaded files in ".\sp-500-stocks" (use force=True to force download)


# Extraction



In [35]:
import pandas as pd
import numpy as np
import os

stock_data = pd.read_csv("C:/Users/Omama/Desktop/sp-500-stocks/sp500_stocks.csv")
stock_data.value_counts()
#company_data = pd.read_csv("C:/Users/Omama/Desktop/sp-500-stocks/sp500_companies.csv")

Date        Symbol  Adj Close   Close       High        Low         Open        Volume   
2010-01-04  A       20.084949   22.389128   22.625179   22.267525   22.453505   3815561.0    1
2019-12-27  MET     43.059624   50.830002   51.340000   50.799999   51.340000   2578200.0    1
            MDLZ    49.610081   55.279999   55.330002   54.799999   54.930000   4229300.0    1
            MCO     229.768723  239.300003  240.309998  238.190002  239.809998  551100.0     1
            MCK     136.506088  139.440002  139.919998  137.860001  139.339996  570300.0     1
                                                                                            ..
2015-02-27  VRSN    64.019997   64.019997   64.900002   63.970001   64.599998   1097100.0    1
            VRSK    69.294281   71.809998   72.879997   71.750000   72.639999   813200.0     1
            VMC     76.624062   83.000000   83.190002   82.209999   82.389999   852700.0     1
            VLO     42.117359   61.689999   62.740002  

# Data Analysis

In [3]:
#print(stock_data.head()) #change the date type
#print(company_data.head()) #make a new col of name, remove desc and ebidta, change employees in int and remove.0

print(stock_data.info())
print(company_data.info())

#print(stock_data.describe())
#print(company_data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1859088 entries, 0 to 1859087
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Date       object 
 1   Symbol     object 
 2   Adj Close  float64
 3   Close      float64
 4   High       float64
 5   Low        float64
 6   Open       float64
 7   Volume     float64
dtypes: float64(6), object(2)
memory usage: 113.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Exchange             503 non-null    object 
 1   Symbol               503 non-null    object 
 2   Shortname            503 non-null    object 
 3   Longname             503 non-null    object 
 4   Sector               503 non-null    object 
 5   Industry             503 non-null    object 
 6   Currentprice         503 non-null    float64
 7   Marketcap            503 non-

### Converting the data types

In [4]:
#assesing the data types for both the tables
print(stock_data.dtypes)
print(company_data.dtypes)

#converting the "Date" column from the stock_data to the datetime format
print(stock_data["Date"].isnull().sum())
stock_data["Date"] = pd.to_datetime(stock_data["Date"])
print(stock_data["Date"].dtypes)

Date          object
Symbol        object
Adj Close    float64
Close        float64
High         float64
Low          float64
Open         float64
Volume       float64
dtype: object
Exchange                object
Symbol                  object
Shortname               object
Longname                object
Sector                  object
Industry                object
Currentprice           float64
Marketcap                int64
Ebitda                 float64
Revenuegrowth          float64
City                    object
State                   object
Country                 object
Fulltimeemployees      float64
Longbusinesssummary     object
Weight                 float64
dtype: object
0
datetime64[ns]


### Checking and replacing/dropping the null values

In [5]:
#checking for the stock_data's null values
print(stock_data.isnull().sum())

#checking for the combinations of the "Symbol" and "Year" that produces all the nulls for the additive data
nulls = stock_data[["Volume", "Adj Close", "Close", "High", "Low", "Open"]].isnull()
stock_data["Year"] = stock_data["Date"].dt.year
null_data = stock_data.loc[nulls.any(axis=1), ["Symbol", "Year"]]

#dropping the null values
stock_data.dropna(subset=["Volume", "Adj Close", "Close", "High", "Low", "Open"], inplace=True)


Date             0
Symbol           0
Adj Close    94879
Close        94879
High         94879
Low          94879
Open         94879
Volume       94879
dtype: int64


In [6]:
#checking for the company_data's null values
print(company_data.isnull().sum())

#checking for the countries having the null states because the dataset has 19 null states
print(company_data.loc[company_data["State"].isnull(), ["City", "State", "Country"]])

#filling the null states with a conditional "No State" value
company_data["State"].fillna("No State", inplace=True)

#checking the null revenuegrowth
print(company_data.loc[company_data["Revenuegrowth"].isnull(), "Shortname"])
print(company_data.loc[(company_data["Shortname"] == "Truist Financial Corporation") | (company_data["Shortname"] == "Western Digital Corporation"), ["Revenuegrowth", "Shortname"]])

#replacing the revenuegrowth by the mean of the revenue growth of the companies lying in the same respective cities of both
print(company_data.loc[(company_data["Shortname"] == "Truist Financial Corporation") | (company_data["Shortname"] == "Western Digital Corporation"), ["Revenuegrowth", "Shortname", "Country"]]) #checking the country for the nulls, which happnes to be the US
mean_rev_growth_USA = company_data.loc[company_data["Country"] == "United States", "Revenuegrowth"].mean()
company_data["Revenuegrowth"].fillna(mean_rev_growth_USA, inplace=True)


Exchange                0
Symbol                  0
Shortname               0
Longname                0
Sector                  0
Industry                0
Currentprice            0
Marketcap               0
Ebitda                 29
Revenuegrowth           2
City                    0
State                  19
Country                 0
Fulltimeemployees       5
Longbusinesssummary     0
Weight                  0
dtype: int64
             City State         Country
34         Woking   NaN  United Kingdom
35         Dublin   NaN         Ireland
80         Zurich   NaN     Switzerland
84         Dublin   NaN         Ireland
121        Swords   NaN         Ireland
122        Dublin   NaN         Ireland
161     Eindhoven   NaN     Netherlands
197          Cork   NaN         Ireland
208  Schaffhausen   NaN     Switzerland
214      Pembroke   NaN         Bermuda
257  Schaffhausen   NaN     Switzerland
280        London   NaN  United Kingdom
334        Dublin   NaN         Ireland
345     Sin

### Dropping duplicates

In [7]:
#checking for the duplicates 
print(stock_data[stock_data.duplicated()])
print(company_data[company_data.duplicated()])


Empty DataFrame
Columns: [Date, Symbol, Adj Close, Close, High, Low, Open, Volume, Year]
Index: []
Empty DataFrame
Columns: [Exchange, Symbol, Shortname, Longname, Sector, Industry, Currentprice, Marketcap, Ebitda, Revenuegrowth, City, State, Country, Fulltimeemployees, Longbusinesssummary, Weight]
Index: []


# Data Transformation

### Dropping Columns

In [8]:
#dropping irrelevant columns from the company_data
company_data.drop(columns=["Longname", "Ebitda", "Fulltimeemployees", "Longbusinesssummary", "Sector"], inplace=True)


### Renaming columns

In [9]:
#renaming columns in the stock_data
stock_data.rename(columns={"Close":"Price_at_Closing", "Adj Close":"Dividends", "High":"Max_Value", "Low":"Min_Value", "Open":"Price_at_Opening"}, inplace=True)
print(stock_data.columns)

#renaming columns in the company_data
company_data.rename(columns={"Shortname":"Name"}, inplace=True)
print(company_data.columns)

Index(['Date', 'Symbol', 'Dividends', 'Price_at_Closing', 'Max_Value',
       'Min_Value', 'Price_at_Opening', 'Volume', 'Year'],
      dtype='object')
Index(['Exchange', 'Symbol', 'Name', 'Industry', 'Currentprice', 'Marketcap',
       'Revenuegrowth', 'City', 'State', 'Country', 'Weight'],
      dtype='object')


### Rounding Values

In [28]:
#rounding the values
stock_data = stock_data.round(2)
company_data = company_data.round(2)
print(company_data)

    Exchange Symbol                     Name                        Industry  \
0        NMS   AAPL               Apple Inc.            Consumer Electronics   
1        NMS   MSFT    Microsoft Corporation       Software - Infrastructure   
2        NMS   NVDA       NVIDIA Corporation                  Semiconductors   
3        NMS   AMZN         Amazon.com, Inc.                 Internet Retail   
4        NMS   GOOG            Alphabet Inc.  Internet Content & Information   
..       ...    ...                      ...                             ...   
498      NMS   PARA         Paramount Global                   Entertainment   
499      NYQ    BWA          BorgWarner Inc.                      Auto Parts   
500      NYQ    IVZ              Invesco Ltd                Asset Management   
501      NMS   ETSY               Etsy, Inc.                 Internet Retail   
502      NYQ   BBWI  Bath & Body Works, Inc.                Specialty Retail   

     Currentprice      Marketcap  Reven

# Migration to PostgreSQL

In [24]:
!pip install psycopg2-binary sqlalchemy


In [25]:
from sqlalchemy import create_engine

db_user = 'postgres'
db_password = 'abs123'
db_host = 'localhost'
db_port = '5432'
db_name = 'dailystocks'

#Connection string
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

#SQLAlchemy engine
engine = create_engine(connection_string)


In [29]:
stock_data.to_sql("stocks", engine, if_exists="append", index=False)
#company_data.to_sql("companies", engine, if_exists="append", index=False)

print("done")

done


In [32]:
print(company_data.value_counts())

Exchange  Symbol  Name                             Industry                                  Currentprice  Marketcap     Revenuegrowth  City        State  Country        Weight
BTS       CBOE    Cboe Global Markets, Inc.        Financial Data & Stock Exchanges          211.81        22162528256    0.07          Chicago     IL     United States  0.0       1
NYQ       IT      Gartner, Inc.                    Information Technology Services           499.57        38496616448    0.06          Stamford    CT     United States  0.0       1
          KKR     KKR & Co. Inc.                   Asset Management                          118.06        108852264960   0.12          New York    NY     United States  0.0       1
          KIM     Kimco Realty Corporation (HC)    REIT - Retail                             23.37         15753998336    0.13          Jericho     NY     United States  0.0       1
          KEYS    Keysight Technologies Inc.       Scientific & Technical Instruments        14